In [7]:
pip install numpy pandas matplotlib scikit-learn torch optuna statsmodels prophet

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

In [9]:
# Generate multivariate time-series data
def generate_time_series(n_steps=1500):
    time = np.arange(n_steps)

    trend = time * 0.01
    seasonal = 10 * np.sin(2 * np.pi * time / 50)
    noise = np.random.normal(0, 1, n_steps)

    x1 = trend + seasonal + noise
    x2 = 0.5 * x1 + np.random.normal(0, 0.5, n_steps)
    x3 = np.sin(time / 10) + np.random.normal(0, 0.3, n_steps)

    data = np.vstack([x1, x2, x3]).T
    return pd.DataFrame(data, columns=["x1", "x2", "x3"])

df = generate_time_series()
df.head()

,x1,x2,x3
0,0.896128,0.660893,0.307015
1,2.159979,1.767569,0.320155
2,3.768882,2.072570,0.532698
3,4.226339,2.115229,0.381057
4,5.921325,2.932837,0.696476


In [10]:
# Scale data & create sequences
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

def create_sequences(data, seq_len=30):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len, 0])  # predict x1
    return np.array(X), np.array(y)

SEQ_LEN = 30
X, y = create_sequences(scaled, SEQ_LEN)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [11]:
# PyTorch Dataset & Loader
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TimeSeriesDataset(X_test, y_test), batch_size=32)

In [15]:
# Attention-based LSTM
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_out):
        weights = torch.softmax(self.attn(lstm_out), dim=1)
        context = torch.sum(weights * lstm_out, dim=1)
        return context

In [16]:
# LSTM + Attention Model
class LSTMAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.attn = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        context = self.attn(lstm_out)
        return self.fc(context).squeeze()

In [17]:
# Train the deep learning model
model = LSTMAttention(input_dim=3, hidden_dim=64)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 20

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for Xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(Xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.0802
Epoch 2, Loss: 0.0517
Epoch 3, Loss: 0.0491
Epoch 4, Loss: 0.0353
Epoch 5, Loss: 0.0281
Epoch 6, Loss: 0.0263
Epoch 7, Loss: 0.0247
Epoch 8, Loss: 0.0207
Epoch 9, Loss: 0.0163
Epoch 10, Loss: 0.0138
Epoch 11, Loss: 0.0140
Epoch 12, Loss: 0.0125
Epoch 13, Loss: 0.0112
Epoch 14, Loss: 0.0114
Epoch 15, Loss: 0.0101
Epoch 16, Loss: 0.0087
Epoch 17, Loss: 0.0073
Epoch 18, Loss: 0.0044
Epoch 19, Loss: 0.0027
Epoch 20, Loss: 0.0023


In [24]:
# Evaluate (MAE, RMSE, MAPE)
model.eval()
preds, actuals = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        p = model(Xb)
        preds.extend(p.numpy())
        actuals.extend(yb.numpy())

mae = mean_absolute_error(actuals, preds)
rmse = np.sqrt(mean_squared_error(actuals, preds))
mape = np.mean(np.abs((np.array(actuals) - np.array(preds)) / np.array(actuals))) * 100

print("LSTM + Attention")
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

LSTM + Attention
MAE: 0.06633099842639197
RMSE: 0.07888344228914733
MAPE: 10.480027


In [25]:
# Baseline 1 — SARIMA
train_series = df["x1"][:split]
test_series = df["x1"][split:]

sarima = SARIMAX(train_series, order=(2,1,2), seasonal_order=(1,1,1,50))
sarima_fit = sarima.fit(disp=False)

sarima_pred = sarima_fit.forecast(len(test_series))

print("SARIMA RMSE:", np.sqrt(mean_squared_error(test_series, sarima_pred)))

SARIMA RMSE: 1.0442279462083095


In [26]:
# Baseline 2 — Prophet
prophet_df = df.copy() # Start with a copy to avoid modifying original df
# Add the 'ds' column as datetime objects
prophet_df['ds'] = pd.to_datetime(pd.date_range(start='2023-01-01', periods=len(prophet_df)))
# Rename 'x1' to 'y' for Prophet
prophet_df.rename(columns={'x1': 'y'}, inplace=True)
# Reorder columns to have 'ds' and 'y' first, followed by other features if needed
prophet_df = prophet_df[['ds', 'y', 'x2', 'x3']]

train_p = prophet_df.iloc[:split]
test_p = prophet_df.iloc[split:]

m = Prophet()
m.fit(train_p[["ds", "y"]])

future = m.make_future_dataframe(periods=len(test_p), include_history=False)
forecast = m.predict(future)

prophet_pred = forecast["yhat"]

print("Prophet RMSE:", np.sqrt(mean_squared_error(test_p["y"], prophet_pred)))

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Prophet RMSE: 7.66009725071163


In [27]:
# Final comparison table
results = pd.DataFrame({
    "Model": ["LSTM+Attention", "SARIMA", "Prophet"],
    "RMSE": [
        rmse,
        np.sqrt(mean_squared_error(test_series, sarima_pred)),
        np.sqrt(mean_squared_error(test_p["y"], prophet_pred))
    ]
})

print(results)

            Model      RMSE
0  LSTM+Attention  0.078883
1          SARIMA  1.044228
2         Prophet  7.660097
